In [5]:
from google.colab import files
uploaded = files.upload()


Saving compas-scores-two-years.xlsx to compas-scores-two-years.xlsx


In [31]:
import pandas as pd
import io
compas_scores = pd.read_excel(io.BytesIO(uploaded['compas-scores-two-years.xlsx']))


In [33]:
compas_scores[(compas_scores["race"]=="Caucasian")]

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_case_number,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,r_case_number,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,violent_recid,is_violent_recid,vr_case_number,vr_charge_degree,vr_offense_date,vr_charge_desc,type_of_assessment,decile_score.1,score_text,screening_date,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
6,8,edward riddle,edward,riddle,2014-02-19,Male,1974-07-23,41,25 - 45,Caucasian,0,6,0,0,14,-1.0,2014-02-18 05:08:24,2014-02-24 12:18:30,14002304CF10A,2014-02-18,NaT,1.0,F,Possession Burglary Tools,1,14004485CF10A,(F2),0.0,2014-03-31,Poss of Firearm by Convic Felo,2014-03-31,2014-04-18,NaN,0,NaN,NaN,NaT,NaN,Risk of Recidivism,6,Medium,2014-02-19,Risk of Violence,2,Low,2014-02-19,2014-03-31,2014-04-18,14,5,40,1,1
8,10,elizabeth thieme,elizabeth,thieme,2014-03-16,Female,1976-06-03,39,25 - 45,Caucasian,0,1,0,0,0,-1.0,2014-03-15 05:35:34,2014-03-18 04:28:46,14004524MM10A,2014-03-15,NaT,1.0,M,Battery,0,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaN,0,NaN,NaN,NaT,NaN,Risk of Recidivism,1,Low,2014-03-16,Risk of Violence,1,Low,2014-03-16,2014-03-15,2014-03-18,0,2,747,0,0
9,13,bo bradac,bo,bradac,2013-11-04,Male,1994-06-10,21,Less than 25,Caucasian,0,3,0,0,1,428.0,2015-01-06 03:55:34,2015-01-07 03:38:44,13000017CF10A,2012-12-31,NaT,308.0,F,Insurance Fraud,1,15002891MM10A,(M1),0.0,2015-01-06,Battery,2015-01-06,2015-01-07,NaN,1,15000258CF10A,(F2),2015-01-06,Aggrav Battery w/Deadly Weapon,Risk of Recidivism,3,Low,2013-11-04,Risk of Violence,5,Medium,2013-11-04,2015-01-06,2015-01-07,1,0,428,1,1
10,14,benjamin franc,benjamin,franc,2013-11-26,Male,1988-06-01,27,25 - 45,Caucasian,0,4,0,0,0,-1.0,2013-11-25 06:31:06,2013-11-26 08:26:57,13016402CF10A,2013-11-25,NaT,1.0,F,"Poss 3,4 MDMA (Ecstasy)",0,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaN,0,NaN,NaN,NaT,NaN,Risk of Recidivism,4,Low,2013-11-26,Risk of Violence,4,Low,2013-11-26,2013-11-25,2013-11-26,0,0,857,0,0
12,16,kortney coleman,kortney,coleman,2013-01-01,Female,1978-08-22,37,25 - 45,Caucasian,0,1,0,0,0,0.0,2013-01-01 03:28:03,2013-01-02 01:12:19,13000053MM10A,2013-01-01,NaT,0.0,M,Battery,0,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaN,0,NaN,NaN,NaT,NaN,Risk of Recidivism,1,Low,2013-01-01,Risk of Violence,1,Low,2013-01-01,2013-01-01,2013-01-02,0,1,1186,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7192,10972,matt munoz,matt,munoz,2013-09-12,Male,1984-03-22,32,25 - 45,Caucasian,0,2,0,0,0,-1.0,2013-09-11 08:56:37,2013-09-12 07:30:00,13012859CF10A,2013-09-11,NaT,1.0,F,Pos Cannabis W/Intent Sel/Del,0,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaN,0,NaN,NaN,NaT,NaN,Risk of Recidivism,2,Low,2013-09-12,Risk of Violence,1,Low,2013-09-12,2013-09-11,2013-09-12,0,0,932,0,0
7194,10976,arleen martin,arleen,martin,2014-12-19,Female,1985-08-14,30,25 - 45,Caucasian,0,1,0,0,2,0.0,2014-12-19 04:46:47,2014-12-20 08:52:23,14017860MM10A,2014-12-19,NaT,0.0,M,Battery,1,15015652CF10A,(F3),1.0,2015-12-05,Felony Battery (Dom Strang),2015-12-06,2015-12-07,NaN,1,15015652CF10A,(F3),2015-12-05,Felony Battery (Dom Strang),Risk of Recidivism,1,Low,2014-12-19,Risk of Violence,1,Low,2014-12-19,2014-12-19,2014-12-20,2,1,351,1,1
7199,10982,austin harris,austin,harris,2013-10-01,Male,1992-07-07,23,Less than 25,Caucasian,0,8,0,0,0,-1.0,2013-09-30 10:10:28,2013-10-03 07:24:16,13013703CF10A,2013-09-30,NaT,1.0,F,Possession Of Methamphetamine,0,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaN,0,NaN,NaN,NaT,NaN,Risk of Recidivism,8,High,2013-10-01,Risk of Violence,6,Medium,2013-10-01,2013-09-30,2013-10-03,0,2,913,0,0
7205,10990,christopher tun,christopher,tun,2013-05-28,Male,1

In [43]:
df = compas_scores[(compas_scores["race"]=="African-American") | (compas_scores["race"]=="Caucasian")]
df["race"] = df["race"].astype('category').cat.codes

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [44]:
df

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_case_number,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,r_case_number,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,violent_recid,is_violent_recid,vr_case_number,vr_charge_degree,vr_offense_date,vr_charge_desc,type_of_assessment,decile_score.1,score_text,screening_date,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,0,0,3,0,0,0,-1.0,2013-01-26 03:45:27,2013-02-05 05:36:53,13001275CF10A,2013-01-26,NaT,1.0,F,Felony Battery w/Prior Convict,1,13009779CF10A,(F3),NaN,2013-07-05,Felony Battery (Dom Strang),NaT,NaT,NaN,1,13009779CF10A,(F3),2013-07-05,Felony Battery (Dom Strang),Risk of Recidivism,3,Low,2013-01-27,Risk of Violence,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,0,0,4,0,1,4,-1.0,2013-04-13 04:58:34,2013-04-14 07:02:04,13005330CF10A,2013-04-13,NaT,1.0,F,Possession of Cocaine,1,13011511MM10A,(M1),0.0,2013-06-16,Driving Under The Influence,2013-06-16,2013-06-16,NaN,0,NaN,NaN,NaT,NaN,Risk of Recidivism,4,Low,2013-04-14,Risk of Violence,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,0,0,8,1,0,1,NaN,NaT,NaT,13000570CF10A,2013-01-12,NaT,1.0,F,Possession of Cannabis,0,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaN,0,NaN,NaN,NaT,NaN,Risk of Recidivism,8,High,2013-01-13,Risk of Violence,6,Medium,2013-01-13,NaT,NaT,1,0,1174,0,0
6,8,edward riddle,edward,riddle,2014-02-19,Male,1974-07-23,41,25 - 45,1,0,6,0,0,14,-1.0,2014-02-18 05:08:24,2014-02-24 12:18:30,14002304CF10A,2014-02-18,NaT,1.0,F,Possession Burglary Tools,1,14004485CF10A,(F2),0.0,2014-03-31,Poss of Firearm by Convic Felo,2014-03-31,2014-04-18,NaN,0,NaN,NaN,NaT,NaN,Risk of Recidivism,6,Medium,2014-02-19,Risk of Violence,2,Low,2014-02-19,2014-03-31,2014-04-18,14,5,40,1,1
8,10,elizabeth thieme,elizabeth,thieme,2014-03-16,Female,1976-06-03,39,25 - 45,1,0,1,0,0,0,-1.0,2014-03-15 05:35:34,2014-03-18 04:28:46,14004524MM10A,2014-03-15,NaT,1.0,M,Battery,0,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaN,0,NaN,NaN,NaT,NaN,Risk of Recidivism,1,Low,2014-03-16,Risk of Violence,1,Low,2014-03-16,2014-03-15,2014-03-18,0,2,747,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7207,10994,jarred payne,jarred,payne,2014-05-10,Male,1985-07-31,30,25 - 45,0,0,2,0,0,0,-1.0,2014-05-09 10:01:33,2014-05-10 08:28:12,14006477CF10A,2014-05-09,NaT,1.0,M,Possess Cannabis/20 Grams Or Less,1,15013710CF10A,(F3),1.0,2015-10-21,Possession of Cannabis,2015-10-22,2015-10-22,NaN,0,NaN,NaN,NaT,NaN,Risk of Recidivism,2,Low,2014-05-10,Risk of Violence,2,Low,2014-05-10,2015-10-22,2015-10-22,0,0,529,1,1
7208,10995,raheem smith,raheem,smith,2013-10-20,Male,1995-06-28,20,Less than 25,0,0,9,0,0,0,-1.0,2013-10-19 11:17:15,2013-10-20 08:13:06,13014650CF10A,2013-10-19,NaT,1.0,F,Possession of Cocaine,0,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaN,0,NaN,NaN,NaT,NaN,Risk of Recidivism,9,High,2013-10-20,Risk of Violence,9,High,2013-10-20,2014-04-07,2014-04-27,0,0,169,0,0
7209,10996,steven butler,steven,butler,2013-11-23,Male,1992-07-17,23,Less than 25,0,0,7,0,0,0,-1.0,2013-11-22 05:18:27,2013-11-24 02:59:20,13016249CF10A,2013-11-22,NaT,1.0,F,Deliver Cannabis,0,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaN,0,NaN,NaN,NaT,NaN,Risk of Recidivism,7,Medium,2013-11-23,Risk of Violence,5,Medium,2013-11-23,2013-11-22,2013-11-24,0,1,860,0,0
7210,10997,malcolm simmons,malcolm,simmons,2014-02-01,Male,1993-03-25,23,Less than 25,0,0,3,0,0,0,-1.0,2014-01-31 07:13:54,2014-02-02 04:0

In [84]:
features = (df[["race", "juv_fel_count", "decile_score", "juv_misd_count", "juv_other_count", "priors_count", "is_violent_recid"]]).to_numpy()
target = (df[["two_year_recid"]]).to_numpy()

In [46]:
def alg3(X, e):
  X1={}
  k=len(set(X[e]))
  for x in set(X[e]): 
    index = X.index
    condition = (X[e] == x)
    indices = index[condition]
    indices_list = indices.tolist()
    X1[x] = indices_list
  return X1

In [90]:
def alg4(Xi, race):
  G = len(Xi["race"]==race)
  Per = (len(Xi[(df["race"]==race) & (Xi["two_year_recid"]==1)])) / (len(Xi[(Xi["race"]==race)]))
  Per2 = (len(Xi[(df["race"]!=race) & (Xi["two_year_recid"]==1)])) / (len(Xi[(Xi["race"]!=race)]))
  Pe = (Per + Per2) / 2
  return G*abs(Per - Pe)

In [93]:
len(target)

6150

In [108]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np
import math

def lps(X, y):
  X_train, X_test, y_train, y_test = train_test_split(X, y.reshape(len(y)), test_size=0.3,random_state=109)
  clf = LogisticRegression(random_state=42)
  clf.fit(X_train, y_train)
  pp = clf.predict_proba(X)
  X = np.append(X, pp[:, 0].reshape(len(pp), 1), axis=1)
  X = np.append(X, pp[:, 1].reshape(len(pp), 1), axis=1)
  
  # Append X and y to make following manipulations easier
  X = np.append(X, y, axis=1)

  # Delete African Americans closest to the boundary predicted "1" and replicate African Americans closest to the boundary predicted "0"
  X0 = X[X[:, 0]==0,]
  tc = len(X0)
  delta_aa = math.floor(0.5*alg4(df, 0))
  count1 = len(X0[:, -1][X0[:, -1]>=0.5])
  count0 = len(X0[:, -2][X0[:, -2]>=0.5])
  indices1 = np.argsort(X0[:, -1])[tc-count1:tc]
  indices0 = np.argsort(X0[:, -2])[tc-count0:tc]
  delete = indices1[0:delta_aa]
  replicate = indices0[0:delta_aa]
  rep = [1]*tc
  for i in replicate:
    rep[i]=2
  for i in delete:
    rep[i]=0
  X0 = np.repeat(X0, repeats = rep, axis=0)

  # Delete Caucasians closest to the boundary predicted "0" and replicate Caucasians closest to the boundary predicted "1"
  X1 = X[X[:, 0]==1,]
  tc = len(X1)
  delta_aa = math.floor(0.5*alg4(df, 1))
  count1 = len(X1[:, -1][X1[:, -1]>=0.5])
  count0 = len(X1[:, -2][X1[:, -2]>=0.5])
  indices1 = np.argsort(X1[:, -1])[tc-count1:tc]
  indices0 = np.argsort(X1[:, -2])[tc-count0:tc]
  replicate = indices1[0:delta_aa]
  delete = indices0[0:delta_aa]
  rep = [1]*tc
  for i in replicate:
    rep[i]=2
  for i in delete:
    rep[i]=0
  X1 = np.repeat(X1, repeats = rep, axis=0)
  X = np.append(X0, X1, axis=0)
  return X


In [109]:
output = lps(features, target)
X_final = np.delete(output, -1, axis=1)
y_final = output[:, -1]

In [116]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3,random_state=109)
clf = LogisticRegression(random_state=12, max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [117]:
metrics.accuracy_score(y_test, y_pred)

0.7053376906318083